In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import time

In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [3]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/


In [4]:
"""
name_projects = []
name_projects.append('project/output_FJSJ_17-02/')
name_projects.append('project/output_FJSJ_17-03/')
name_projects.append('project/output_FJSJ_17-04/')
name_projects.append('project_repartition_v3.0/output_repar_v9.1_02-01/')
name_projects.append('project_repartition_v4.0/output_repar_v9.2_01-01/')
"""
name_projects = []
name_projects.append('project/output_regular_v9.5_3x7/')
name_projects.append('project/output_regular_v9.5_5x13/')
name_projects.append('project/output_regular_v9.5_7x19/')
name_projects.append('project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/')

In [5]:
name_project_sample = 'project_repartition_v4.0/output_repar_v9.5_01--10-16Hz/'
dir_project_sample = dir_project_workspace + name_project_sample
filename = dir_project_sample+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
key_subworks = info_basic['key_subworks']
key_subworks

['1--08-03',
 '2--09-03',
 '3--10-03',
 '4--11-03',
 '5--12-03',
 '6--13-03',
 '7--14-03',
 '8--15-03',
 '9--16-03',
 '10--18-03',
 '12--20-03',
 '13--21-03',
 '15--23-03',
 '17--25-03',
 '18--26-03',
 '19--27-03',
 '20--28-03',
 '21--29-03',
 '22--30-03',
 '23--31-03',
 '25--33-03',
 '26--34-03',
 '28--36-03',
 '30--41-03',
 '31--42-03',
 '33--46-03',
 '34--47-03',
 '35--48-03',
 '36--52-03',
 '37--53-03',
 '38--07-04',
 '39--08-04',
 '40--09-04',
 '41--10-04',
 '42--12-04',
 '43--13-04',
 '44--17-04',
 '45--20-04',
 '46--21-04',
 '48--23-04',
 '49--24-04',
 '50--25-04',
 '51--27-04',
 '52--28-04',
 '53--29-04',
 '54--30-04',
 '55--31-04',
 '56--32-04',
 '57--33-04',
 '58--34-04',
 '59--35-04',
 '60--36-04',
 '63--47-04',
 '65--49-04',
 '66--52-04',
 '67--53-04',
 '68--08-05',
 '69--12-05',
 '70--13-05',
 '71--14-05',
 '72--16-05',
 '75--22-05',
 '76--24-05',
 '77--26-05',
 '78--27-05',
 '79--28-05',
 '80--29-05',
 '81--30-05',
 '84--35-05',
 '85--36-05',
 '86--37-05',
 '88--39-05',
 

In [6]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_stations_all = stainfo['latitude'].tolist() 
lon_stations_all = stainfo['longitude'].tolist()

In [7]:
stations_alls = {}
lat_stations_alls = {}
lon_stations_alls = {}
lat_centroid_alls = {}
lon_centroid_alls = {}
for name_project in name_projects:
    dir_project = dir_project_workspace + name_project
    filename = dir_project+'Basic_info.yml'
    with open(filename, 'r', encoding='utf-8') as f:
        info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
    dir_partition = dir_project + 'partition/'
    stations = {}
    lat = {}
    lon = {}
    lat_centroid = {}
    lon_centroid = {}
    for key in info_basic['key_subworks']:
        filepath = dir_partition + str(key) + '.txt'
        stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
        stations[key] = stations_this.tolist()
        lat[key] = lat_stations_this.astype(float)
        lon[key] = lon_stations_this.astype(float)
        lat_centroid[key] = np.mean(lat[key])
        lon_centroid[key] = np.mean(lon[key])

    stations_alls[name_project] = stations
    lat_stations_alls[name_project] = lat
    lon_stations_alls[name_project] = lon
    lat_centroid_alls[name_project] = lat_centroid
    lon_centroid_alls[name_project] = lon_centroid
    

In [8]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [9]:
dir_image = dir_project_workspace + 'compare_relative_error/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)

In [10]:
def find_nearset(key_subwork,name_project):
	global name_project_sample
	global key_subworks

	lon_this = lon_centroid_alls[name_project_sample][key_subwork]
	lat_this = lat_centroid_alls[name_project_sample][key_subwork]

	lon_stations_all = []
	lat_stations_all = []
	
	lon_stations_all = list(lon_centroid_alls[name_project].values())
	lat_stations_all = list(lat_centroid_alls[name_project].values())
	key_all = list(lat_centroid_alls[name_project].keys())

	dist = np.sqrt((np.array(lon_stations_all)-lon_this)**2+(np.array(lat_stations_all)-lat_this)**2)
	indx = np.argsort(dist)
	#print(indx)
	key_nearset = key_all[indx[0]]

	return str(key_nearset)

In [11]:
c_map = 'jet'
v_min = 0.1
v_max = None
xlim_f = [1,25]
for key_subwork in key_subworks:
    print('start ',str(key_subwork),' ',str(key_subworks.index(key_subwork)+1),'/',len(key_subworks))
    fig,ax = plt.subplots(2,len(name_projects),figsize=(5*len(name_projects),9))
    for name_project in name_projects:
        key_this_find = find_nearset(key_subwork,name_project)
        #print(name_project)

        stations = stations_alls[name_project]
        lat_stations = lat_stations_alls[name_project]
        lon_stations = lon_stations_alls[name_project]


        col_index = name_projects.index(name_project)
        dir_project = dir_project_workspace + name_project
        filename = dir_project+'Basic_info.yml'
        with open(filename, 'r', encoding='utf-8') as f:
            info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
        dir_ds = dir_project + info_basic['rdir_ds']

        #print('key_this_find: ',key_this_find)

        stalist = stations[key_this_find]
        lat_this = lat_stations[key_this_find]
        lon_this = lon_stations[key_this_find]
        nsta = len(stalist)

        ax[1][col_index]=plotlib.plot_area(ax[1][col_index],lon_stations_all,lat_stations_all,lon_this,lat_this)
        for i in range(len(faults)):
            ax[1][col_index].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
        ax[1][col_index].scatter(np.mean(lon_this),np.mean(lat_this),c='g',marker = '^',s=50)
        
        title0 = name_project.split('/')[-2] + ' ' +str(key_this_find)
        c = np.linspace(info_basic['fj_c_min'],info_basic['fj_c_max'],info_basic['fj_c_num'])
        ds = h5py.File(dir_ds+'ds_'+str(key_this_find)+'.h5', 'r')
        ds_remove = ds['ds_remove'][0]
        ds_linear = ds['ds_linear'][0]
        ds_linear = plotlib.smooth_ds(ds_linear)
        ds_remove = plotlib.smooth_ds(ds_remove)
        f = ds['f'][:]
        ax[0][col_index] = plotlib.plot_fj(ax[0][col_index] ,ds_remove,title0,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
        ax[0][col_index] .set_xlim(xlim_f)
        #ax[1][col_index] = plotlib.plot_fj(ax[1][col_index] ,ds_linear,title0,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
        #ax[1][col_index] .set_xlim(xlim_f)
    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'.png',dpi=60)
    plt.close()
    #time.sleep(7)

start  1--08-03   1 / 466
start  2--09-03   2 / 466
start  3--10-03   3 / 466
start  4--11-03   4 / 466
start  5--12-03   5 / 466
start  6--13-03   6 / 466
start  7--14-03   7 / 466
start  8--15-03   8 / 466
start  9--16-03   9 / 466
start  10--18-03   10 / 466
start  12--20-03   11 / 466
start  13--21-03   12 / 466
start  15--23-03   13 / 466
start  17--25-03   14 / 466
start  18--26-03   15 / 466
start  19--27-03   16 / 466
start  20--28-03   17 / 466
start  21--29-03   18 / 466
start  22--30-03   19 / 466
start  23--31-03   20 / 466
start  25--33-03   21 / 466
start  26--34-03   22 / 466
start  28--36-03   23 / 466
start  30--41-03   24 / 466
start  31--42-03   25 / 466
start  33--46-03   26 / 466
start  34--47-03   27 / 466
start  35--48-03   28 / 466
start  36--52-03   29 / 466
start  37--53-03   30 / 466
start  38--07-04   31 / 466
start  39--08-04   32 / 466
start  40--09-04   33 / 466
start  41--10-04   34 / 466
start  42--12-04   35 / 466
start  43--13-04   36 / 466
start  44-